<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 00 - Scenario Overview and System Setup

In this set of tutorials you'll work with an end-to-end scenario that uses SQL Server 2019's big data clusters to solve real-world problems. 


## Wide World Importers

**TODO:** Describe company and use-case

## Running these Tutorials

- You can read through the output of these completed tutorials if you wish - or:

- You can follow along with the steps you see in these tutorials by copying the code into a SQL Query window and Spark Notebook using the Azure Data Studio tool, or you can click here to download these Jupyter Notebooks and run them in Azure Data Studio for a hands-on experience.
 
- If you would like to run the tutorials, you'll need a SQL Server 2019 big data cluster and the client tools installed. If you want to set up your own cluster, <a href="https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-get-started?view=sqlallproducts-allversions" target="_blank">click this reference and follow the steps you see there for the server and tools you need</a>.

- You will need to have the following: 
    - Your **Knox Password**
    - The **Knox IP Address**
    - The `sa` **Username** and **Password** to your Master Instance
    - The **IP address** to the SQL Server big data cluster Master Instance 
    - The **name** of your big data cluster

For a complete workshop on SQL Server 2019's big data clusters, <a href="https://github.com/Microsoft/sqlworkshops/tree/master/sqlserver2019bigdataclusters" target="_blank">check out this resource</a>.

## Copy Database backups to the SQL Server 2019 big data cluster Master Instance

**TODO:** describe curl, etc. Code for file transfer, describe restore process

In [2]:
!md d:\temp

!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/WWI.bak" -o d:\temp\WWI.bak
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/WWIDW.bak" -o d:\temp\WWIDW.bak

!kubectl cp d:/temp/WWI.bak mssql-master-pool-0:/var/opt/mssql/data -c mssql-server -n <ReplaceWithClusterName>
!kubectl cp d:/temp/WWIDW.bak mssql-master-pool-0:/var/opt/mssql/data -c mssql-server -n <ReplaceWithClusterName>

## Copy Exported Data to Storage Pool

**TODO:** Explain curls to hdfs

In [ ]:
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/product_review_data?op=MKDIRS"
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/adventureworks_export?op=MKDIRS"

!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/product_reviews.csv" -o d:\temp\product_reviews.csv
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/customers.csv" -o d:\temp\customers.csv
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/web_clickstreams.csv" -o d:\temp\web_clickstreams.csv
!curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/stockitemholdings.csv" -o d:\temp\stockitemholdings.csv

!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/product_review_data/product_reviews.csv?op=create&overwrite=true" -H "Content-Type: application/octet-stream" -T "product_reviews.csv"
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/adventureworks_export/customers.csv?op=create&overwrite=true" -H "Content-Type: application/octet-stream" -T "customers.csv.csv"
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/adventureworks_export/web_clickstreams.csv?op=create&overwrite=true" -H "Content-Type: application/octet-stream" -T "web_clickstreams.csv"
!curl -i -L -k -u root:<ReplacewithKnoxPassword> -X PUT "https://<ReplaceWithKnoxEndpoint>:30443/gateway/default/webhdfs/v1/adventureworks_export/stockitemholdings.csv?op=create&overwrite=true" -H "Content-Type: application/octet-stream" -T "stockitemholdings.csv"

## Next Step: Working with the SQL Server 2019 big data cluster Master Instance

**TODO:** Add Link